# 构建一个提取链
- 项目教程地址：
    - https://python.langchain.com/docs/tutorials/extraction/
- DeepSeek在LangChain中的集成包文档：
    - https://python.langchain.com/api_reference/deepseek/chat_models/langchain_deepseek.chat_models.ChatDeepSeek.html


- 在这个教程中， 我们将会使用大模型的tool_calling功能来从非结构化的文本中提取结构化的信息，这个教程也会展示如何使用少样本提示（few-shot prompting）来提升性能。
- **此教学需要langchain-core版本在0.3.20以上，并且需要大模型工具具备tool calling功能**
    - 以下是如何查看自己的langchain-core版本的两种方法

In [1]:
import langchain_core
print(langchain_core.__version__)

0.3.44


In [ ]:
pip show langchain-core

In [ ]:
# 更新版本
pip install --upgrade langchain-core

## 环境配置
1. 需要Jupyter Notebook
2. 安装
    - pip install --upgrade langchain-core
3. LangSmith配置见以下代码：
    - 我分享在Github中的Langchain项目下的其他项目有解释，0基础请从头看起。

In [3]:
import getpass
import os 
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

 ········


## 数据模式定义
- 首先我们需要定义我们想要从文本中提取数据的结构是怎样的
- 我们将会使用Pydantic来定义一个样例模型来提取个人信息，代码如下：

In [8]:
from typing import Optional 
from pydantic import BaseModel, Field 

class Person(BaseModel):
    """Information about a person."""
    
    # ^ 这是Person实体的文档字符串
    # 这个文档字符串会被发送给大模型作为Person数据结构的描述内容
    # 这将帮助优化文本提取结果
    
    # 提示
    # 1. 下面的每个字段都是可选项 - 这允许大模型不提取该字段。 
        # 提高灵活性，在文本没有提供该字段的时候允许大模型不提取该字段。
    # 2. 每个字段都包含描述属性 - 大模型在提取内容时会参考描述内容
        # 好的描述能够帮助优化提取结果
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

- Schema（模式）为什么使用Optional模块？
    - Pydantic 使用字段的类型注解来验证输入数据。如果注解是 name: str，Pydantic 会要求输入必须是字符串，None 会导致验证失败。
    - 用 Optional[str]，Pydantic 明确知道这个字段允许 None，并在解析时接受 None 作为合法值。
- 为Schema写好注释非常重要，同时确保在没有相关信息时不会让大模型强制输出结果，这样才能够使得结果更加准确。

# 2.提取器
- 基于创建好的数据输出模式创建信息提取器

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 定义一个个性化的提示词来提供指示和任何额外的上下文。
# 1) 你可以添加样例到提示词模版中来优化提取质量。
# 2) 引入额外的参数把上下文考虑进去。（例如文档的Metadata属性）


prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm."
            "Only extract relevant information from the text."
            "If you do not know the value of an attribute asked to extract,"
            "return null for the attribute's value.",
        ),
        # 请查看How-to文档来优化参考样例的性能
        ("human","{text}"),
    ]
)

- 我们需要使用支持function或者tool calling的大模型

In [6]:
import getpass
import os 
# 前置需要进入deepseek官网注册并生成API_KEY，记得保存好，否则丢失后需要重新生成。
# 这一行检查程序是否已经有 Deepseek API 密钥。如果没有，就需要用户手动输入。
if not os.getenv("DEEPSEEK_API_KEY"):
    os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("Enter your DeepSeek API key: ")

# 配置接口参数，创建语言模型的实例
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(
    model="deepseek-chat", #指定使用的模型
    temperature=0, #控制生成文本的随机性
    max_tokens=None, #生成的最大token数限制
    timeout=None, #请求超时时间
    max_retries=2, #请求失败时的最大重试次数
    # other params...
)

Enter your DeepSeek API key:  ········


In [10]:
# 构建模型数据输出实例，按照Person的数据结构输出结果文本
structured_llm = llm.with_structured_output(schema=Person)

In [15]:
text = "Vincent is 6 feet tall and has black hair."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Person(name='Vincent', hair_color='black', height_in_meters='1.83')

# 3.抽取多个实体
- 在大部分情况下，你应该需要提取很多实体，而不仅仅只是一个
- 这可以通过在pydantic中嵌套大模型来实现

In [16]:
from typing import List, Optional 

from pydantic import BaseModel, Field 

class Person(BaseModel):
    """Information about a person."""
    
    # 实体Person的文档字符串
    # 这个文档字符串会被发给大模型作为Person数据输出结构的描述，这能够帮助优化数据提取结果
    
    # 提示：
    # 1. 每个字段都是可选的 -- 这允许大模型不提取这个字段，在某些信息不存在的情况下
    # 2. 每个字段都有描述 -- 这个描述会被大模型采用
    # 好的描述能够帮助优化提取结果
    
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )
    
class Data(BaseModel):
    """Extract data about people."""
    
    # Creates a model so that we can extract multiple entities.
    people: List[Person]

1. 为什么导入 List？
    - List 来自 typing，用于指定 people 字段的类型为 列表。
    - 在 Data 类中，people 字段是 List[Person]，表示它是一个由 多个 Person 对象 组成的列表。
    - 这意味着提取结果可以包含 多个人的信息，而不仅仅是单个 Person 实例。
2. people 字段的写法目的，以及逻辑是什么？
- 目的：
    - people 这个字段的目的是 允许提取多个 Person 实体，而不仅仅是一个。这样，当数据中包含多个不同的人的信息时，可以将所有人的信息结构化地存储在 people 这个列表中。
- 逻辑：
    - people: List[Person] 说明 Data 类的 people 字段是一个包含 多个 Person 实例的列表。
    - pydantic 会根据 Person 这个数据结构，对 people 里的每一个 Person 进行 验证 和 格式化，确保数据符合定义的格式。

In [17]:
structured_llm = llm.with_structured_output(schema=Data)
text = "My name is Jeff, my hair is balck and i am 6 feet tall. Anna has the same color hair as me. Vincent is 1.8 meters and has red hair."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Data(people=[Person(name='Jeff', hair_color='black', height_in_meters='1.83'), Person(name='Anna', hair_color='black', height_in_meters=None), Person(name='Vincent', hair_color='red', height_in_meters='1.8')])

- 当输出模式适应多实体文本数据提取，这也允许大模型提取非实体如果文本中没有相关信息，最终大模型会输出一个空的列表。
- 这通常是件好事。它允许为实体指定必需的属性，而不一定强制模型检测到该实体。

# 4.参考样例

- 大语言模型的表现可以被引导通过少样本提示（few-shot prompting）.这样聊天模型就可以拿到一系列成对的输入和回答的模型，这些模型展示了我们想要的结果。
- 例如，我们可以传输

In [18]:
messages = [
    {"role": "user", "content": "2 🦜 2"},
    {"role": "assistant", "content": "4"},
    {"role": "user", "content": "2 🦜 3"},
    {"role": "assistant", "content": "5"},
    {"role": "user", "content": "3 🦜 4"},
]

response = llm.invoke(messages)
print(response.content)

7


1. 代码解读：
    1. messages 是一个对话历史的列表，其中包含了用户 ("role": "user") 和助手 ("role": "assistant") 之间的交互。
    2. 在前几个示例中，用户输入 "2 🦜 2"，助手输出 "4"，用户输入 "2 🦜 3"，助手输出 "5"。
    3. 这个 🦜（鹦鹉）符号可能被用作自定义运算符，模型需要从上下文中推断其含义，看起来它代表的是加法（2 + 2 = 4, 2 + 3 = 5）。
    4. response = llm.invoke(messages) 让 LLM 继续这个对话，针对 "3 🦜 4" 生成一个合理的输出，预计它会返回 "7"，因为之前的模式表明 🦜 可能意味着加法。
2. 目的：
    1. 这个写法的目的是提供一些示例对话，让模型学会如何处理 🦜 这个符号的含义，从而在新输入 "3 🦜 4" 时推理出正确的结果。

- 不同的大模型服务商对消息排序会有不同的需求。一些接受如下的模型消息排序，例如：
    - 用户消息
    - 工具调用的AI消息
    - 返回的工具消息 
   另外一些模型会要求包含了特定顺序的最终的AI指令。
- LangChain有一个实用性的方法叫做 tool_example_to_message，这个方法会生成一个大部分大模型服务商都会接受的顺序。它简化了结构化少样本的生成，仅仅只要求所调用工具的Pydantic定义的数据输出结构。
- 让我们来尝试一下。我们可以把一堆输入字符串和想要的Pydantic对象转变成有序的消息，这样大模型就能顺利生成这个结果。 LangChain将会在底层将too calls整理成服务商所需要的格式。

In [20]:
from langchain_core.utils.function_calling import tool_example_to_messages

examples = [
    (
        "The ocean is vast and blue. It's more than 20,000 feet deep.",
        Data(people=[]),
    ),
    (
        "Fiona traveled far from France to Spain.",
        Data(people=[Person(name="Fiona", height_in_meters=None, hair_color=None)]),
    ),

]

messages = []

for txt, tool_call in examples:
    if tool_call.people:
        # 对于一部分大模型，最终的结果是可选的
        ai_response = "Detected people."
    else:
        ai_response = "Detected no people."
    messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))
    
    

/var/folders/py/wsw9dv2n5g70rl8sm3pxxjg40000gn/T/ipykernel_9873/1222595702.py:23: LangChainBetaWarning: The function `tool_example_to_messages` is in beta. It is actively being worked on, so the API may change.
  messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))


In [24]:
messages

[HumanMessage(content="The ocean is vast and blue. It's more than 20,000 feet deep.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'bc4ae030-defb-437c-87ea-90354ad53db9', 'type': 'function', 'function': {'name': 'Data', 'arguments': '{"people":[]}'}}]}, response_metadata={}, tool_calls=[{'name': 'Data', 'args': {'people': []}, 'id': 'bc4ae030-defb-437c-87ea-90354ad53db9', 'type': 'tool_call'}]),
 ToolMessage(content='You have correctly called this tool.', tool_call_id='bc4ae030-defb-437c-87ea-90354ad53db9'),
 AIMessage(content='Detected no people.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Fiona traveled far from France to Spain.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '674c04fa-33a6-45cd-ada0-da1473877eb7', 'type': 'function', 'function': {'name': 'Data', 'arguments': '{"people":[{"name":"Fiona","hair_color":null,"height_in_m

**代码解释**
1. for txt, tool_call in examples:
    - 这里的 examples 是一个包含多个元组的列表。每个元组里有两个元素，第一个是文本（字符串），第二个是结构化数据（一个 Data 对象）。
    - 在循环中，txt 被赋值为文本内容，而 tool_call 被赋值为对应的 Data 对象。
2. if tool_call.people:
    - 这里的判断逻辑是检查 tool_call 对象中的 people 列表是否非空。
    - 如果 people 列表中存在至少一个 Person 实例，则条件为 True，表示文本中检测到了人物；否则条件为 False。
3. tool_example_to_messages：
    - 这是从 langchain_core.utils.function_calling 模块中导入的一个函数，用于将一个工具调用示例转换成一系列格式化的消息。
    - 它会把给定的文本、工具调用输出（这里是一个 Data 对象的列表）以及 AI 响应转换为符合 LangChain 消息格式的列表。
4. messages.extend(...)：
    - messages 是一个 Python 列表，.extend() 是列表的方法，用于将另一个列表中的所有元素添加到 messages 列表末尾。
    - 这样，所有转换后的消息都会被追加到 messages 列表中，形成一个完整的消息序列。

5. 综合来说，这段代码遍历示例数据，为每个文本与其对应的结构化数据（Data 对象）生成一组消息，并根据是否检测到人物设置不同的 AI 响应，最后将所有生成的消息追加到 messages 列表中。

In [21]:
for message in messages:
    message.pretty_print()

================================ Human Message =================================

The ocean is vast and blue. It's more than 20,000 feet deep.
================================== Ai Message ==================================
Tool Calls:
  Data (bc4ae030-defb-437c-87ea-90354ad53db9)
 Call ID: bc4ae030-defb-437c-87ea-90354ad53db9
  Args:
    people: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

Detected no people.
================================ Human Message =================================

Fiona traveled far from France to Spain.
================================== Ai Message ==================================
Tool Calls:
  Data (674c04fa-33a6-45cd-ada0-da1473877eb7)
 Call ID: 674c04fa-33a6-45cd-ada0-da1473877eb7
  Args:
    people: [{'name': 'Fiona', 'hair_color': None, 'height_in_meters': None}]
================================

In [22]:
message_no_extraction = {
    "role": "user",
    "content": "The solar system is large, but earth has only 1 moon.",
}

structured_llm = llm.with_structured_output(schema=Data)
structured_llm.invoke([message_no_extraction])

Data(people=[Person(name='Earth', hair_color=None, height_in_meters=None)])

In [23]:
structured_llm.invoke(messages + [message_no_extraction])

Data(people=[])

- 以上可以对比有样本学习的情况和无样本学习的情况，有样本的情况下大模型学会了识别语句中是否有人的元素，无样本的情况下大模型直接将物质作为人的信息进行了输出。